In [1]:
import os
import csv
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options




# LOADING DATA

In [2]:
current_dataset = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/latest_transfermarkt_data_money_fixed.csv')

new_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/NEW TransferMarkt Market Values.csv', low_memory=False)


In [3]:
countrycodes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/countries_and_codes.csv')


# FUNCTIONS

In [4]:
from difflib import SequenceMatcher
def get_best_match(query, choices):
    return max(choices, key=lambda choice: SequenceMatcher(None, query, choice).ratio())


# Function to remove accents
def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countrycodes.loc[countrycodes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None
# Apply the function to create Player IDs in the salaries DataFrame

# LOOKING AT DATA

In [5]:
current_dataset.head(5)

,Name,Nationality,Position,Market Value,Season,League,Team,Team 1 Code,Birthday,Age,PlayerID,Accents Removed
0,Joe Hart,England,Goalkeeper,16000000.0,2013,EPL,Manchester City,EN,"Apr 19, 1987",27,"Joe Hart_Apr 19, 1987_England",Joe Hart
1,Costel Pantilimon,Romania,Goalkeeper,2500000.0,2013,EPL,Manchester City,RO,"Feb 01, 1987",27,"Costel Pantilimon_Feb 01, 1987_Romania",Costel Pantilimon
2,Richard Wright,England,Goalkeeper,250000.0,2013,EPL,Manchester City,EN,"Nov 05, 1977",36,"Richard Wright_Nov 05, 1977_England",Richard Wright
3,Vincent Kompany,Belgium,Centre-Back,35000000.0,2013,EPL,Manchester City,BE,"Apr 10, 1986",28,"Vincent Kompany_Apr 10, 1986_Belgium",Vincent Kompany
4,Matija Nastasic,Serbia,Centre-Back,21000000.0,2013,EPL,Manchester City,RS,"Mar 28, 1993",21,"Matija Nastasic_Mar 28, 1993_Serbia",Matija Nastasic


In [6]:
new_data.head(5)

,Name,Nationality,Position,Market Value,Season,Club,Birthday,Age,PlayerID
0,Sergio Romero,Argentina,Goalkeeper,€4.00m,2013,CA Boca Juniors,"Feb 22, 1987",25.0,"Sergio Romero_Feb 22, 1987_Argentina"
1,Mariano Andújar,Argentina,Goalkeeper,€4.00m,2013,Retired,"Jul 30, 1983",29.0,"Mariano Andújar_Jul 30, 1983_Argentina"
2,Agustín Orión,Argentina,Goalkeeper,€1.00m,2013,Retired,"Jun 26, 1981",31.0,"Agustín Orión_Jun 26, 1981_Argentina"
3,Óscar Ustari,Argentina,Goalkeeper,€800k,2013,Without Club,"Jul 3, 1986",26.0,"Óscar Ustari_Jul 3, 1986_Argentina"
4,Ezequiel Garay,Argentina,Centre-Back,€14.00m,2013,Retired,"Oct 10, 1986",26.0,"Ezequiel Garay_Oct 10, 1986_Argentina"


## WORKING WITH NEW DATA

### Edit market value

In [7]:
value_rows_with_currency_symbol = new_data[new_data['Market Value'].str.contains('€', na=False)].reset_index()

In [8]:
for i in range(0, len(new_data)):
#for i in range(0, 3):

    #index_correct = value_data.at[i, 'index']

    money_column = new_data.at[i, 'Market Value']

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)

        new_data.at[i, 'Market Value'] = millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        new_data.at[i, 'Market Value'] = millinum
    elif '-' in money_column:
        new_data.at[i, 'Market Value'] = 0
    else:
        print(money_column)


### Accents Removed

In [9]:
new_data['Accents Removed'] = new_data['Name'].apply(lambda x: unidecode(x))


### Adding Country ID

In [10]:
notfoundset = []
for i in range(0, len(new_data)):
    this_country = new_data.at[i, 'Nationality']

    if(this_country.endswith('U-20')):
        this_country = this_country.split(' U-20')[0]

    if(this_country == 'Czech Republic'):
        this_country = 'Czech'

    if(this_country == 'Bosnia-Herzegovina'):
        this_country = 'Bosnia'
    
    if(this_country == 'Brunei Darussalam'):
        this_country = 'Brunei'
    
    if(this_country == 'China'):
        this_country = 'China PR'
    
    if(this_country == 'Curacao'):
        this_country = 'Curaçao'
    
    if(this_country == 'Korea, North'):
        this_country = 'North Korea'
    
    if(this_country == 'Korea, South'):
        this_country = 'South Korea'
    
    if(this_country == 'Macao'):
        this_country = 'Macau'
    
    if(this_country == 'Neukaledonien'):
        this_country = 'New Caledonia'
    
    if(this_country == 'Neukaledonien'):
        this_country = 'New Caledonia'
    
    if(this_country == 'Northern Ireland'):
        this_country = 'North. Ireland'
    
    if(this_country == 'Saint-Martin'):
        this_country = 'St. Martin'
    
    if(this_country == 'Sao Tome and Principe'):
        this_country = 'Sao Tome'
    
    if(this_country == 'St. Kitts &amp; Nevis'):
        this_country = 'Saint Kitts and Nevis'
    
    if(this_country == 'St. Lucia'):
        this_country = 'Saint Lucia'
    
    if(this_country == 'United States'):
        this_country = 'USA'

    if(this_country == 'Democratic Republic of the Congo'):
        this_country = 'DR Congo'

    if(this_country == 'Republic of the Congo'):
        this_country = 'Congo R'

    if(this_country == 'Turks and Caicos Islands'):
        this_country = 'Turks and Caicos Iss'

    if(this_country == 'United States Virgin Islands'):
        this_country = 'US Virgin Islands'
    

    code_df = countrycodes[countrycodes['Country'] == this_country].reset_index()
    if(len(code_df) == 0):
        #print(f"{this_country} not found")
        notfoundset.append(this_country)
    elif(len(code_df) == 1):
        0==0
        #values.at[i, 'Team 1 Code'] = code_df.at[0, ' Code']

set(notfoundset)


{'British Virgin Islands', 'Tahiti'}

In [11]:


# Add a new column 'Accents Removed'

data = {'Country': ['British Virgin Islands'], ' Code': ['BVI'], 'Accents Removed': ['British Virgin Islands']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)

data = {'Country': ['Tahiti'], ' Code': ['TI'], 'Accents Removed': ['Tahiti']}
one_row_dataframe = pd.DataFrame(data)
countrycodes = pd.concat([countrycodes, one_row_dataframe], ignore_index=True)

countrycodes['Accents Removed'] = countrycodes['Country'].apply(remove_accents)



In [11]:
new_data['Team 1 Code'] = new_data['Nationality'].apply(get_country_code)


### Add League (blank for now)

In [12]:
new_data['League'] = ''

In [13]:
new_data.rename(columns={'Club': 'Team'}, inplace=True)


In [14]:
desired_order = ['Name', 'Nationality', 'Position', 'Market Value', 'Season', 'League', 'Team', 'Team 1 Code', 'Birthday', 'Age', 'PlayerID', 'Accents Removed']
new_data = new_data[desired_order]

# CHECKING PLAYER IDs

### First - fixing birthdays and IDs

### Fixing N/A Birthdays

In [15]:
for index, row in new_data.iterrows():
    if pd.isnull(row['Birthday']):
        if not pd.isnull(row['Age']):
            # If Age is known, calculate Birthday
            birth_year = int(row['Season']) - int(row['Age'])
            birthday = f"Jan 01, {birth_year}"
        else:
            # If Age is not known, set to Jan 01, 2000
            birthday = "Jan 01, 2000"
        
        # Update Birthday column
        new_data.at[index, 'Birthday'] = birthday
        
        # Update PlayerID
        if "N/A" in row['PlayerID']:
            new_data.at[index, 'PlayerID'] = f"{row['Name']}_{birthday}_{row['Nationality']}"
        else:
            # Find the first instance of "_" and insert Birthday after that
            pos = row['PlayerID'].find('_')
            new_data.at[index, 'PlayerID'] = row['PlayerID'][:pos+1] + birthday + '_' + row['PlayerID'][pos+1:]

### Resolving edge cases

In [16]:
new_data.at[10074, 'Birthday'] = 'Jan 01, 1985'
new_data.at[10074, 'Age'] = 29
new_data.at[10074, 'PlayerID'] = 'Mykyta Kamenyuka_Jan 01, 1985_Ukraine'

new_data.at[10119, 'Birthday'] = 'Jan 01, 1985'
new_data.at[10119, 'Age'] = 30
new_data.at[10119, 'PlayerID'] = 'Mykyta Kamenyuka_Jan 01, 1985_Ukraine'

new_data.at[11089, 'Birthday'] = 'Jan 01, 1983'
new_data.at[11089, 'Age'] = 32
new_data.at[11089, 'PlayerID'] = 'Robert Almer_Jan 01, 1983_Austria'

new_data.at[49739, 'Birthday'] = 'Jan 01, 1987'
new_data.at[49739, 'Age'] = 27
new_data.at[49739, 'PlayerID'] = "Ibrahima Sy_Jan 01, 1987_Mauritania"

### Fixing single digit date for birthday

In [ ]:
# test_BD = "Feb 1, 1987"
# day = test_BD.split()[1].split(',')[0]

# #f"{test_BD[:3]} 0{test_BD[4:]}"

# f"{test_BD[:3]} 0{day}{test_BD[5:]}"

#new_data.at[index, 'PlayerID'].replace(original_date, new_birthday)

In [17]:
for index, row in new_data.iterrows():
    # Split the day and handle cases with a comma
    day = row['Birthday'].split()[1].split(',')[0]
    
    # Check if the day is a single digit
    if len(day) == 1:
        # If it's a single digit, add a leading zero
        new_birthday = f"{row['Birthday'][:3]} 0{day}{row['Birthday'][5:]}"
        
        # Update the 'Birthday' column with the new date
        new_data.at[index, 'Birthday'] = new_birthday
        
        # Update the 'PlayerID' column with the new date
        original_date = row['Birthday']
        new_data.at[index, 'PlayerID'] = new_data.at[index, 'PlayerID'].replace(original_date, new_birthday)

### Comparing player IDs

### Venn Diagram method

In [ ]:
tracking_DF = pd.DataFrame()

In [28]:
for i in range(0, len(new_data)): #len(new_data)
    this_player_ID = new_data.at[i, 'PlayerID']
    #print(f'id # {i}: ', this_player_ID)
    newdatasetplayer = new_data[new_data['PlayerID'] == this_player_ID]

    current_Dataset_player = current_dataset[current_dataset['PlayerID'] == this_player_ID]

    if(len(current_Dataset_player) == 0):
        #they are not in the CURRENT DATASET - CASE C.
        #IF ANY OF THE ROWS ARE 0 MV
            #THEN DO A FOR LOOP 
        current_dataset = pd.concat([current_dataset, newdatasetplayer], ignore_index=True)
    else:
        0==0
        # #TASK 1 - ADD SEASONS THAT CURRENT DATASET DOESN'T HAVE
        # curr_data_seasons = set(current_Dataset_player['Season'].unique())
        # new_data_seasons = set(newdatasetplayer['Season'].unique())

        # #print('   seasons, current data: ', curr_data_seasons, 'new data: ', new_data_seasons)

        # seasons_diff = (new_data_seasons - curr_data_seasons)
        # seasons_diff = list(seasons_diff)
        # if(len(seasons_diff) == 0):
        #     0==0
        #     #print(f'   TASK 1 NO. new dataset doesn\'t have any seasons current data does not have.')
        # else:
        #     #print('   TASK 1 YA')
        #     for j in range(0, len(seasons_diff)):
        #         #print('       adding seasons from new data: ', seasons_diff[j])
        #         thatseason = newdatasetplayer[newdatasetplayer['Season'] == seasons_diff[j]]

        #         current_dataset = pd.concat([current_dataset, thatseason], ignore_index=True)

        # #TASK 2 - REPLACE SEASONS WHERE CURRENT DATASET HAS MV == 0
        # seasonschanged = 0
        # for k in range(0, len(curr_data_seasons)):
            
        #     curr_season = list(curr_data_seasons)[k]
        #     this_season_curr_data = current_Dataset_player[current_Dataset_player['Season'] == curr_season].reset_index()

        #     if(len(this_season_curr_data) >= 2):
        #         0==0
        #         #print('    there was more than 1 record for the same season in the current data: ', curr_season)
        #         #the goal here is take the max value of the set. but if they're all 0, act accordingly
        #     elif(len(this_season_curr_data) == 1):   
        #         0==0
        #         if(this_season_curr_data.at[0, 'Market Value'] == 0):
        #         #if the market value is equal to zero, substitute the data from the new dataset in
        #             new_data_same_season =  new_data[new_data['Season'] == curr_season].reset_index()
        #             market_value_new_data_same_season = new_data_same_season.at[0, 'Market Value']

        #             correct_season_index = this_season_curr_data.at[0, 'index']
        #             #print('    TASK 2 YA. replacing market value with: ', market_value_new_data_same_season)
        #             current_dataset.at[correct_season_index, 'Market Value'] = market_value_new_data_same_season
        #             seasonschanged += 1
        #         else:
        #             0==0 #that means the market value is not zero. 
        #             #so we do nothing
        # if(seasonschanged == 0):
        #     0==0
        #     #print('    Task 2 NO')


In [31]:
current_dataset.to_csv('most_updated_transfermarkt_dataset.csv', index=False)


#### INDEX USING NEW DATASET

In [ ]:
for i in range(0, 10):#127
    this_player_ID = new_data.at[i, 'PlayerID']

#### INDEXING USING CURRENT DATASET

In [ ]:
#for i in range(0, len(current_dataset)):
for i in range(0, 10):#127

    this_player_ID = current_dataset.at[i, 'PlayerID']

    counter_singlematch = 0
    counter_multi_match = 0
    counter_nomatches = 0

    #if that ID is in the dataset 
    new_data_player = new_data[new_data['PlayerID'] == this_player_ID]

    if(len(new_data_player['Name'].value_counts()) == 1):
        #that nationality + birthday isn't in the dataset
        counter_singlematch+=1
    elif(len(new_data_player['Name'].value_counts()) >= 2):
        #more than 1 name in this set???
        counter_multi_match += 1
    elif(len(new_data_player) == 0):
        counter_nomatches += 1

        #this_name = current_dataset.at[i, 'Name']
        this_bd = current_dataset.at[i, 'Birthday']
        this_natl = current_dataset.at[i, 'Nationality']

        subset_bd_natl = new_data[(new_data['Nationality']==this_natl) & (new_data['Birthday']==this_bd)]

        if(len(subset_bd_natl) == 0):
            0==0
        else:
            print(i, 'bd natl match', current_dataset.at[i, 'Name'], subset_bd_natl['Name'].unique())



In [ ]:
# for i in range(0, len(new_data)):
#     this_player_ID = new_data.at[i, 'PlayerID']

#     #if that is in the dataset 
#     current_Dataset_player = current_dataset[current_dataset['PlayerID'] == this_player_ID]

#     if(len(current_Dataset_player) == 0):
#         #that ID isn't in the dataset
#         0==0
#     elif(len(current_Dataset_player['Name'].value_counts()) >= 2):
#         #more than 1 name in this set???
#         0==0